## 补充
* 权值太多，参数过多，需要样本数量大
* 引入局部感受野

###  卷积层就是对图像像素进行特征提取，多个不同卷积核产生不同特征图，增加深度，减少参数
### 池化层就是降低分辨率，缩小尺寸，减少参数，加快计算

### CONV(卷积层-RELU(激活函数)-POOL(池化层)... FC(全连接层) 如下图


## tensorboard报错 ：InvalidArgumentError: You must feed a value for placeholder tensor 'inputs/y_input
https://github.com/HuangCongQing/TensorFlow/issues/8




![](https://cdn.nlark.com/yuque/0/2019/png/232596/1572371566724-41591b3b-8b49-454d-a4df-d2e3ba33e6cc.png)


## 进入tensorboard查看
```
F:  // win系统首先进入所在盘

tensorboard --logdir=F:\Github\TensorFlow\深度学习框架Tensorflow学习与应用\logs\log-6.1
```


In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# 载入数据集
mnist = input_data.read_data_sets("../MNIST_data", one_hot=True) 


Extracting ../MNIST_data\train-images-idx3-ubyte.gz
Extracting ../MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data\t10k-labels-idx1-ubyte.gz


In [21]:
batch_size = 128
n_batch = mnist.train.num_examples // batch_size
max_step = 1000
keep_ = 0.8
log_dir = "logs/log-6.1"

# 生成权重
def weight_variable(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1),name='W')

# 生成偏差
def bias_vairable(shape):
    return tf.Variable(tf.constant(0.1, shape=shape),name='b')

# 记录变量
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME',name='conv2d')
        
def conv_layer(input_tensor, weight_shape, layer_name, act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = weight_variable(weight_shape)
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_vairable([weight_shape[-1]])
            variable_summaries(biases)
        with tf.name_scope('conv_comput'):
            preactivate = conv2d(input_tensor,weights) + biases
        with tf.name_scope('activate'):
            activations = act(preactivate)
        return activations

def linear_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_vairable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('linear_comput'):
            preactivate = tf.matmul(input_tensor,weights) + biases
        with tf.name_scope('activate'):
            activations = act(preactivate)
        return activations

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME',name='Max_pool')

with tf.name_scope('Input'):
    x = tf.placeholder(tf.float32,[None,784],name='input_x')
    with tf.name_scope('Input_reshape'):
        x_image = tf.reshape(x,[-1,28,28,1],name='x-image')
        input_summery = tf.summary.image('input',x_image,10)
    y = tf.placeholder(tf.float32,[None,10],name='input_y')
    keep_prob = tf.placeholder(tf.float32,name='keep_prob')

# 第一次卷积   28*28*1->28*28*32
conv_layer1 = conv_layer(x_image,[5,5,1,32],'conv_layer1')
# 池化之后变为 14*14*32
with tf.name_scope('Max_pool1'):
    h_pool1 = max_pool_2x2(conv_layer1)

# 第二次卷积 14*14*32->14*14*64
conv_layer2 = conv_layer(h_pool1,[5,5,32,64],'conv_layer2')
# 第二次池化之后变为 7*7*64
with tf.name_scope('Max_pool2'):
    h_pool2 = max_pool_2x2(conv_layer2)

with tf.name_scope('Flatten'):
    flatten_ = tf.reshape(h_pool2,[-1,7*7*64])
    
# 第一个全连接层 7*7*64 - 1024
fc1 = linear_layer(flatten_, 7*7*64, 1024, 'FC1')

with tf.name_scope('Dropput'):
    fc1_drop = tf.nn.dropout(fc1, keep_prob)
    
# 第二个全连接层 1024 - 10
logits = linear_layer(fc1_drop, 1024, 10, 'FC2',act=tf.nn.sigmoid)

with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
    loss_summary = tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

with tf.name_scope('accuracy'):
    prediction = tf.nn.softmax(logits)
    correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    accuracy_summary = tf.summary.scalar('accuracy', accuracy)
# merged = tf.summary.merge_all()
merged = tf.summary.merge([input_summery,loss_summary, accuracy_summary])

def get_dict(train):
    if train:
        xs, ys = mnist.train.next_batch(batch_size)
        k = keep_
    else:
        xs, ys = mnist.test.images, mnist.test.labels
        k = 1.0
    return {x:xs, y:ys, keep_prob: k}

with tf.Session() as sess:
    train_writer = tf.summary.FileWriter(log_dir + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(log_dir + '/test')
    # train_writer.add_graph(sess.graph)#添加graph图
    # test_writer.add_graph(sess.graph)#添加graph图
    
    sess.run(tf.global_variables_initializer())
    
    for i in range(max_step):
        if i%10 == 0:
            summary,acc = sess.run([merged,accuracy], feed_dict=get_dict(False))
            test_writer.add_summary(summary, i)
            print("Step: " + str(i) + ", acc: " + str(acc))
        else:
            summary,_ = sess.run([merged,train_step], feed_dict=get_dict(True))
            train_writer.add_summary(summary,i)
    train_writer.close()
    test_writer.close()

Step: 0, acc: 0.0862
Step: 10, acc: 0.1287
Step: 20, acc: 0.1879
Step: 30, acc: 0.1952
Step: 40, acc: 0.1712
Step: 50, acc: 0.1754
Step: 60, acc: 0.1918
Step: 70, acc: 0.1514
Step: 80, acc: 0.2119
Step: 90, acc: 0.2036
Step: 100, acc: 0.212
Step: 110, acc: 0.2961
Step: 120, acc: 0.3105
Step: 130, acc: 0.3227
Step: 140, acc: 0.2691
Step: 150, acc: 0.3671
Step: 160, acc: 0.3945
Step: 170, acc: 0.4187
Step: 180, acc: 0.4798
Step: 190, acc: 0.5776
Step: 200, acc: 0.5821
Step: 210, acc: 0.6107
Step: 220, acc: 0.6309
Step: 230, acc: 0.6714
Step: 240, acc: 0.6922
Step: 250, acc: 0.7187
Step: 260, acc: 0.7277
Step: 270, acc: 0.7182
Step: 280, acc: 0.7005
Step: 290, acc: 0.722
Step: 300, acc: 0.7339
Step: 310, acc: 0.7177
Step: 320, acc: 0.7613
Step: 330, acc: 0.7285
Step: 340, acc: 0.7483
Step: 350, acc: 0.7324
Step: 360, acc: 0.7528
Step: 370, acc: 0.752
Step: 380, acc: 0.757
Step: 390, acc: 0.7353
Step: 400, acc: 0.7487
Step: 410, acc: 0.7627
Step: 420, acc: 0.7848
Step: 430, acc: 0.8006
Ste

![](https://cdn.nlark.com/yuque/0/2020/png/232596/1596253070053-454f25f5-7025-49a8-bfb9-65b77e4db986.png)